---
layout: post  
---

In a previous post, I found a type-stability issue in a function that I was writing.
The issue is described [here](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-value-type) in the Julia language manual.

In [1]:
import Pkg
pkgs = [
    "BioSequences",
    "Random",
    "BenchmarkTools",
    "Primes"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
k = 3
sequence = BioSequences.randdnaseq(Random.seed!(1), 10)

10nt DNA Sequence:
TCGTCCCAGG

In [3]:
function assess_kmers_1(k, sequence)
    KMER_TYPE = BioSequences.DNAMer{k}
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_1 (generic function with 1 method)

In [5]:
BenchmarkTools.@benchmark assess_kmers_1(k, sequence)

BenchmarkTools.Trial: 
  memory estimate:  2.02 KiB
  allocs estimate:  54
  --------------
  minimum time:     5.360 μs (0.00% GC)
  median time:      5.660 μs (0.00% GC)
  mean time:        6.045 μs (2.48% GC)
  maximum time:     805.995 μs (98.05% GC)
  --------------
  samples:          10000
  evals/sample:     6

In [6]:
@code_warntype assess_kmers_1(k, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_1, false)
  k::Int64
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  KMER_TYPE::Type{BioSequences.Mer{BioSequences.DNAAlphabet{2},_A}} where _A
  kmers::Set{_A} where _A
  @_6::Union{Nothing, Tuple{BioSequences.MerIterResult,Tuple{Int64,Any,Any,Any}}}
  kmer_set::BioSequences.MerIterResult

Body::Set{_A} where _A
1 ─ %1  = BioSequences.DNAMer::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},K} where K, false)
│         (KMER_TYPE = Core.apply_type(%1, k))
│   %3  = Core.apply_type(Main.Set, KMER_TYPE)::Type{Set{_A}} where _A
│         (kmers = (%3)())
│   %5  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %6  = KMER_TYPE::Type{BioSequences.Mer{BioSequences.DNAAlphabet{2},_A}} where _A
│   %7  = (%5)(%6, sequence)::BioSequences.EveryMerIterator{_A,BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}} where _A<:BioSequences.AbstractMer
│         (@_6 = Base.iterate(%7))
│ 

In [7]:
function assess_kmers_2(K::Val{k}, sequence) where k
    KMER_TYPE = BioSequences.DNAMer{k}
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_2 (generic function with 1 method)

In [8]:
BenchmarkTools.@benchmark assess_kmers_2(Val(k), sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     2.530 μs (0.00% GC)
  median time:      2.624 μs (0.00% GC)
  mean time:        2.678 μs (1.24% GC)
  maximum time:     335.687 μs (98.61% GC)
  --------------
  samples:          10000
  evals/sample:     9

In [9]:
@code_warntype assess_kmers_2(Val(k), sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_2, false)
  K::Core.Compiler.Const(Val{3}(), false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  KMER_TYPE::Type{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_6::Union{Nothing, Tuple{BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}},Tuple{Int64,Int64,UInt64,UInt64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = BioSequences.DNAMer::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},K} where K, false)
│         (KMER_TYPE = Core.apply_type(%1, $(Expr(:static_parameter, 1))))
│   %3  = Core.apply_type(Main.Set, KMER_TYPE::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false))::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers

In [10]:
function assess_kmers_3(KMER_TYPE, sequence)
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_3 (generic function with 1 method)

In [11]:
BenchmarkTools.@benchmark assess_kmers_3(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  2.02 KiB
  allocs estimate:  54
  --------------
  minimum time:     5.450 μs (0.00% GC)
  median time:      5.659 μs (0.00% GC)
  mean time:        6.049 μs (2.35% GC)
  maximum time:     784.787 μs (97.96% GC)
  --------------
  samples:          10000
  evals/sample:     6

In [12]:
@code_warntype assess_kmers_3(BioSequences.DNAMer{k}, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_3, false)
  KMER_TYPE::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_5::Union{Nothing, Tuple{BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}},Tuple{Int64,Int64,UInt64,UInt64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = Core.apply_type(Main.Set, KMER_TYPE)::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers = (%1)())
│   %3  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %4  = (%3)(KMER_TYPE, sequence)::BioSequences.EveryMerIterator{BioSequences.Mer{BioSequences.DNAAlphabet{2},3},BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}}
│         (@_5 = Base.iterate(

In [13]:
function assess_kmers_4(KMER_TYPE::Type{BioSequences.Mer{A, K}}, sequence) where {A, K}
    kmers = Set{BioSequences.Mer{A, K}}()
    for kmer_set in BioSequences.each(BioSequences.Mer{A, K}, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_4 (generic function with 1 method)

In [14]:
BenchmarkTools.@benchmark assess_kmers_4(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     430.965 ns (0.00% GC)
  median time:      457.548 ns (0.00% GC)
  mean time:        502.030 ns (6.47% GC)
  maximum time:     20.523 μs (96.97% GC)
  --------------
  samples:          10000
  evals/sample:     199

In [15]:
@code_warntype assess_kmers_4(BioSequences.DNAMer{k}, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_4, false)
  KMER_TYPE::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_5::Union{Nothing, Tuple{BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}},Tuple{Int64,Int64,UInt64,UInt64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = BioSequences.Mer::Core.Compiler.Const(BioSequences.Mer, false)
│   %2  = $(Expr(:static_parameter, 1))::Core.Compiler.Const(BioSequences.DNAAlphabet{2}, false)
│   %3  = Core.apply_type(%1, %2, $(Expr(:static_parameter, 2)))::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
│   %4  = Core.apply_type(Main.Set, %3)::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false

In [16]:
function assess_kmers_5(KMER_TYPE::Type{BioSequences.Mer{A, K}}, sequence) where {A, K}
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_5 (generic function with 1 method)

In [17]:
BenchmarkTools.@benchmark assess_kmers_5(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     441.758 ns (0.00% GC)
  median time:      457.909 ns (0.00% GC)
  mean time:        496.724 ns (6.78% GC)
  maximum time:     21.624 μs (97.36% GC)
  --------------
  samples:          10000
  evals/sample:     198

In [18]:
@code_warntype assess_kmers_5(BioSequences.DNAMer{k}, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_5, false)
  KMER_TYPE::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_5::Union{Nothing, Tuple{BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}},Tuple{Int64,Int64,UInt64,UInt64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = Core.apply_type(Main.Set, KMER_TYPE)::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers = (%1)())
│   %3  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %4  = (%3)(KMER_TYPE, sequence)::BioSequences.EveryMerIterator{BioSequences.Mer{BioSequences.DNAAlphabet{2},3},BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}}
│         (@_5 = Base.iterate(

In [19]:
function assess_kmers_6(::Type{KMER_TYPE}, sequence) where KMER_TYPE <: BioSequences.AbstractMer
    kmers = Set{KMER_TYPE}()
    for kmer_set in BioSequences.each(KMER_TYPE, sequence)
        push!(kmers, BioSequences.canonical(kmer_set.fw))
    end
    return kmers
end

assess_kmers_6 (generic function with 1 method)

In [20]:
BenchmarkTools.@benchmark assess_kmers_6(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     429.462 ns (0.00% GC)
  median time:      455.327 ns (0.00% GC)
  mean time:        496.297 ns (6.95% GC)
  maximum time:     24.325 μs (97.93% GC)
  --------------
  samples:          10000
  evals/sample:     199

In [21]:
@code_warntype assess_kmers_6(BioSequences.DNAMer{k}, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_6, false)
  #unused#::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
  @_5::Union{Nothing, Tuple{BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}},Tuple{Int64,Int64,UInt64,UInt64}}}
  kmer_set::BioSequences.MerIterResult{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─ %1  = Core.apply_type(Main.Set, $(Expr(:static_parameter, 1)))::Core.Compiler.Const(Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}, false)
│         (kmers = (%1)())
│   %3  = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %4  = $(Expr(:static_parameter, 1))::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
│   %5  = (%3)(%4, sequence)::BioSequences.EveryMerIterator{BioSe

In [42]:
function assess_kmers_7(::Type{KMER_TYPE}, sequence) where KMER_TYPE <: BioSequences.AbstractMer
    kmers = Set(BioSequences.canonical(kmer.fw) for kmer in BioSequences.each(KMER_TYPE, sequence))
    return kmers
end

assess_kmers_7 (generic function with 1 method)

In [43]:
BenchmarkTools.@benchmark assess_kmers_7(BioSequences.DNAMer{k}, sequence)

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  5
  --------------
  minimum time:     432.970 ns (0.00% GC)
  median time:      458.080 ns (0.00% GC)
  mean time:        504.432 ns (7.05% GC)
  maximum time:     23.278 μs (97.69% GC)
  --------------
  samples:          10000
  evals/sample:     199

In [44]:
@code_warntype assess_kmers_7(BioSequences.DNAMer{k}, sequence)

Variables
  #self#::Core.Compiler.Const(assess_kmers_7, false)
  #unused#::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
  sequence::BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}
  #22::var"#22#23"
  kmers::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}

Body::Set{BioSequences.Mer{BioSequences.DNAAlphabet{2},3}}
1 ─      (#22 = %new(Main.:(var"#22#23")))
│   %2 = #22::Core.Compiler.Const(var"#22#23"(), false)
│   %3 = BioSequences.each::Core.Compiler.Const(BioSequences.each, false)
│   %4 = $(Expr(:static_parameter, 1))::Core.Compiler.Const(BioSequences.Mer{BioSequences.DNAAlphabet{2},3}, false)
│   %5 = (%3)(%4, sequence)::BioSequences.EveryMerIterator{BioSequences.Mer{BioSequences.DNAAlphabet{2},3},BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}}
│   %6 = Base.Generator(%2, %5)::Base.Generator{BioSequences.EveryMerIterator{BioSequences.Mer{BioSequences.DNAAlphabet{2},3},BioSequences.LongSequence{BioSequences.DNAAlphabet{4}}},var"#22